In [395]:
import pandas as pd
import morph_kgc as mgc
import random
import numpy as np

random.seed(42)
np.random.seed(42)

In [ ]:
# Load the transaction data (source: https://www.kaggle.com/datasets/vipin20/transaction-data)
tx = pd.read_csv('../data/transaction_data.csv', sep=',', encoding='utf-8', dtype={"ItemCode": str})
tx['TransactionTime'] = pd.to_datetime(tx['TransactionTime'].str.replace("IST ", ""), format="%a %b %d %H:%M:%S %Y")

In [397]:
tx.dtypes

UserId                             int64
TransactionId                      int64
TransactionTime           datetime64[ns]
ItemCode                          object
ItemDescription                   object
NumberOfItemsPurchased             int64
CostPerItem                      float64
Country                           object
dtype: object

In [398]:
items = (
    tx
    .loc[(tx['ItemCode'] != '-1') & (tx['UserId'].astype(str) != '-1')]
    .sort_values('TransactionTime', ascending=False)
    .groupby(["ItemCode", "Country"], as_index=False)
    .agg(
        ItemDescription =('ItemDescription', 'first'),
        CostPerItem = ('CostPerItem', 'first'),
        LastTransactionDate = ('TransactionTime', 'first'),
        TransactionCount = ('ItemCode', 'count')
    )
    .assign(processing_item_desc = lambda x: x['ItemDescription'].str.replace(r'[^A-z ]', '', regex=True))
    .assign(is_set = lambda x: x['ItemDescription'].str.contains('(?<!\w)set(?!\w)', case=False, regex=True))
    .drop(columns=['processing_item_desc'])
)
items.head()

,ItemCode,Country,ItemDescription,CostPerItem,LastTransactionDate,TransactionCount,is_set
0,1010331,Channel Islands,DOORMAT 3 SMILEY CATS,10.98,2018-11-04 08:30:00,4,False
1,1010331,EIRE,DOORMAT 3 SMILEY CATS,11.39,2018-12-22 08:37:00,4,False
2,1010331,France,DOORMAT 3 SMILEY CATS,9.78,2019-01-28 07:59:00,2,False
3,1010331,Greece,DOORMAT 3 SMILEY CATS,9.32,2018-04-07 07:39:00,2,False
4,1010331,Spain,DOORMAT 3 SMILEY CATS,10.98,2018-04-23 10:44:00,4,False


In [399]:
# Code below is simply to help find hierarchical attributes (like department)
stopwords = ['AND', 'OF', 'FOR', 'TO', 'THE', 'IN', 'A', 'AN', 'IS', 'AT', 'ON',
             'WITH', 'AS', 'BY', 'FROM', 'THAT', 'THIS', 'WHICH', 'BE', 'OR', '']
kw = (
    items
    .assign(keywords = lambda x: x['ItemDescription'].str.split(' '))
    .explode('keywords')
    .groupby('keywords', as_index=False)
    .agg(word_count=('ItemCode', 'count'))
    .pipe(lambda df: df[~df['keywords'].isin(stopwords)])
    .sort_values('word_count', ascending=False)
)
kw.head()

,keywords,word_count
1687,SET,1952
1566,RED,1403
2046,VINTAGE,1187
138,BAG,1098
1441,PINK,1084


In [400]:
def identify_department(item_description, keyword_mapping = {}):
    for keyword, grouping in keyword_mapping.items():
        if keyword.lower() in item_description.lower():
            return grouping
        
    return "Miscellaneous"


In [401]:
departments_mapping = {
    "CHRISTMAS": "CHRISTMAS",
    "SANTA": "CHRISTMAS",
    "GARLAND": "CHRISTMAS",
    "XMAS": "CHRISTMAS",
    "BIRTHDAY": "BIRTHDAY",
    "HOME": "HOME AND GARDEN",
    "GARDEN": "HOME AND GARDEN",
    "DOORMAT": "HOME AND GARDEN",
    "MIRROR": "HOME AND GARDEN",
    'CANDLE': 'HOME AND GARDEN',
    'LIGHTS': 'HOME AND GARDEN',
    'WALL': 'HOME AND GARDEN',
    'LANTERN': 'HOME AND GARDEN',
    'CHIME': 'HOME AND GARDEN',
    'TRAY': 'HOME AND GARDEN',
    "COASTER": "HOME AND GARDEN",
    "LIGHTBULB": "HOME AND GARDEN",
    "DOORBELL": "HOME AND GARDEN",
    "FRAME": "HOME AND GARDEN",
    "CLOCK": "HOME AND GARDEN",
    "BULB": "HOME AND GARDEN",
    "BATH": "HOME AND GARDEN",
    "PANTRY": "KITCHEN",
    "KITCHEN": "KITCHEN",
    "CAKE": "KITCHEN",
    "TEA": "KITCHEN",
    "EGG": "KITCHEN",
    "BAKING": "KITCHEN",
    "CUTLERY": "KITCHEN",
    "APRON": "KITCHEN",
    "COFFEE": "KITCHEN",
    'TUMBLER': 'KITCHEN',
    'ORANGE SQUEEZER': 'KITCHEN',
    "COOKIE JAR": "KITCHEN",
    "GLASS": "KITCHEN",
    "BOTTLE": "KITCHEN",
    "FELTCRAFT": "HOBBY",
    "PENCILS": "HOBBY",
    "SEWING": "HOBBY",
    "PHOTO": "HOBBY",
    "WARMER": "HOBBY",
    "BOOK": "HOBBY",
    "KNIT": "HOBBY",
    "CARD": "HOBBY",
    'BAG': 'STORAGE',
    'HOLDER': 'STORAGE',
    'STORAGE': 'STORAGE',
    'BOX': 'STORAGE',
    'POLE': 'STORAGE',
    "RACK": "STORAGE",
    'BIN': 'STORAGE',
    'SOCK': 'APPAREL',
    'SHIRT': 'APPAREL',
    'TROUSER': 'APPAREL',
    'PANT': 'APPAREL',
    'CAP': 'APPAREL',
    'SOMBRERO': 'APPAREL',
    'OFFICE': 'OFFICE',
    "SIGN": "OFFICE",
    "PEN": "OFFICE"
}

In [402]:
items['Department'] = items['ItemDescription'].apply(identify_department, keyword_mapping=departments_mapping)
items['Department'].value_counts()

Department
Miscellaneous      7200
KITCHEN            2834
STORAGE            2362
HOME AND GARDEN    2316
HOBBY              1325
CHRISTMAS           939
OFFICE              700
BIRTHDAY            214
APPAREL             112
Name: count, dtype: int64

In [403]:
# For our purposes, we will want each item to have a department assigned, so we shall filter out any items that are not assigned a department
items_final = items.loc[items['Department'] != 'Miscellaneous'].reset_index(drop=True)

In [404]:
manager_mapping = {
    "John": ['OFFICE', 'HOBBY'],
    "Mary": ['CHRISTMAS', 'BIRTHDAY'],
    "Steve": ['KITCHEN'],
    "Anya": ['STORAGE'],
    "Samantha": ['HOME AND GARDEN'],
    "Jackson": ['APPAREL'],
}

# Convert to a pandas DataFrame
manager_mapping_df = pd.DataFrame()
for manager, departments in manager_mapping.items():
    for department in departments:
        manager_mapping_df = pd.concat([manager_mapping_df, pd.DataFrame({'Manager': [manager], 'Department': [department]})], ignore_index=True, axis = 0)

manager_mapping_df = manager_mapping_df.drop_duplicates().reset_index(drop=True)
manager_mapping_df

,Manager,Department
0,John,OFFICE
1,John,HOBBY
2,Mary,CHRISTMAS
3,Mary,BIRTHDAY
4,Steve,KITCHEN
5,Anya,STORAGE
6,Samantha,HOME AND GARDEN
7,Jackson,APPAREL


In [405]:
# Create a dataframe of 50 sales reps and their assigned manager (chosen randomly but deterministically)
sales_reps = pd.DataFrame({
    'SalesRep': [f'SalesRep_{i}' for i in range(50)],
    'Manager': np.random.choice(manager_mapping_df['Manager'], size = 50)
})
sales_reps['Manager'].value_counts()

Manager
Mary        15
Jackson      8
John         8
Steve        7
Samantha     6
Anya         6
Name: count, dtype: int64

In [406]:
# Map Countries to Regions (Thanks Mistral for the mapping)
country_to_region = {
    'Channel Islands': 'Europe',
    'EIRE': 'Europe',
    'France': 'Europe',
    'Greece': 'Europe',
    'Spain': 'Europe',
    'Switzerland': 'Europe',
    'United Kingdom': 'Europe',
    'Germany': 'Europe',
    'Unspecified': 'Other',
    'Australia': 'Asia-Pacific',
    'Portugal': 'Europe',
    'Italy': 'Europe',
    'RSA': 'Other',
    'Singapore': 'Asia-Pacific',
    'United Arab Emirates': 'Middle East',
    'Finland': 'Europe',
    'Netherlands': 'Europe',
    'Austria': 'Europe',
    'Belgium': 'Europe',
    'Cyprus': 'Europe',
    'USA': 'North America',
    'Denmark': 'Europe',
    'Iceland': 'Europe',
    'Sweden': 'Europe',
    'Canada': 'North America',
    'Malta': 'Europe',
    'Norway': 'Europe',
    'Bahrain': 'Middle East',
    'European Community': 'Europe',
    'Lebanon': 'Middle East',
    'Poland': 'Europe',
    'Israel': 'Middle East',
    'Japan': 'Asia-Pacific',
    'Czech Republic': 'Europe',
    'Brazil': 'Other',
    'Lithuania': 'Europe',
    'Saudi Arabia': 'Middle East'
}

items_final['Region'] = items_final['Country'].map(country_to_region).fillna('Other')
items_final['Region'].value_counts()

Region
Europe           9639
Asia-Pacific      578
Middle East       207
Other             189
North America     189
Name: count, dtype: int64

In [407]:
# Randomly assign sales reps a region which has items under the purview of their manager such that each region a manager has sales in has at least one sales rep
regions = items_final['Region'].unique()
sales_reps['Region'] = sales_reps['Manager'].apply(lambda x: manager_mapping_df.loc[manager_mapping_df['Manager'] == x, 'Department'].values[0])
sales_reps['Region'] = sales_reps['Region'].apply(lambda x: random.choice(items_final.loc[items_final['Department'] == x, 'Region'].unique()))

# Randomly determine if sales reps can sell item sets
sales_reps['CanSellSets'] = random.choices([True, False], k=len(sales_reps), weights=[0.8, 0.2])

In [408]:
sales_reps.value_counts('Region')

Region
Europe           15
Middle East      11
Asia-Pacific      9
North America     8
Other             7
Name: count, dtype: int64

In [409]:
sales_reps.value_counts('CanSellSets')

CanSellSets
True     40
False    10
Name: count, dtype: int64

In [410]:
# Save the three dataframes to CSV files
items_final.to_csv('../data/items_final.csv', index=False, encoding='utf-8')
sales_reps.to_csv('../data/sales_reps.csv', index=False, encoding='utf-8')
manager_mapping_df.to_csv('../data/manager_mapping.csv', index=False, encoding='utf-8')

In [411]:
items_final[['Department', 'Region']].drop_duplicates().sort_values(['Department', 'Region'])

,Department,Region
6061,APPAREL,Asia-Pacific
95,APPAREL,Europe
8219,APPAREL,Middle East
6062,APPAREL,North America
8202,APPAREL,Other
2577,BIRTHDAY,Asia-Pacific
518,BIRTHDAY,Europe
3696,BIRTHDAY,Middle East
2837,BIRTHDAY,North America
2812,BIRTHDAY,Other


In [412]:
sales_reps.groupby("Manager").agg(region_ct = ('Region', 'nunique')).reset_index().sort_values('region_ct', ascending=False)

,Manager,region_ct
3,Mary,5
2,John,5
4,Samantha,4
0,Anya,4
5,Steve,4
1,Jackson,3
